In [2]:
import time
import requests
from bs4 import BeautifulSoup
import urllib
from wiktionaryparser import WiktionaryParser
import pandas as pd
import numpy as np
import os
#from pymystem3 import Mystem
#m = Mystem()
import sys
import re
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
def lem_coll(coll):
    
    ws = coll.split(' ')
    
    lem_coll = ''
    
    #preps = ['на', 'в', 'под', 'за']
    
    stop_lemming = False
    w_idx = 0
    while w_idx < len(ws):
        
        word = ws[w_idx]
        
        if word in preps:
            
            stop_lemming = True
            
        if stop_lemming == False:
            
            p = morph.parse(word)[0]
            
            if p.tag.POS == 'NOUN':
                lem_coll += p.inflect({'nomn'}).word + ' '
            
            else:
                lem_coll += p.normal_form + ' '
            
        else:
            
            lem_coll += word + ' '
            
        w_idx += 1

    return ' ' + lem_coll

In [ ]:
def extract_n_grams(directory):
    
    punct = [',', '.', '.', '...', '?', '!', '-', '—', ' – ', '--', '\'\'', ';', ':', '(', ')']

    #REPLACE WITH STOP WORDS FOR QUILL VERSION:
    ignored_words = ['не', 'и', 'или', 'я', 'ты', 'мы', 'вы', 'меня', 'тебя', 'нас', 'вас', 'мне', 'тебе', 'нам', 'вам', 'меня', 'тебя', 'нас', 'вас', '', 'мной', 'тобой', 'нами', 'вами', 'мне', 'тебе', 'нас', 'вас', 'он', 'она', 'оно', 'они', 'oн', 'oнa', 'oнo', 'oни', 'его', 'её', 'его', 'их', 'ему', 'eй', 'ей', 'ему', 'им', 'его', 'её', 'его', 'их', 'им', 'ей', 'ею', 'им', 'ими', 'нём', 'ней', 'нём', 'них']
    
    n_gram_dict = {}
    
    counted_tokens = {}

    doc = ''
    
    stop_idx = 0
    for file in os.listdir(directory):
            filename = os.fsdecode(file)

            if filename.endswith('.txt') and 'ENG' not in filename:

                with open((os.path.join(directory, filename)), 'r') as text:

                        text = text.read()

                        #if stop_idx == 0:
                        doc += text
                    
                stop_idx += 1
                            
                            
    
    doc_sents = nltk.sent_tokenize(doc)

    for sent in doc_sents:
    
        sent_words = nltk.word_tokenize(sent)

        idx = 0
        while idx + 1 <= len(sent_words):

            word = sent_words[idx]
            
            bigrams = []
            trigrams = []
            four_grams = []
            five_grams = []
            six_grams = []
            seven_grams = []
            eight_grams = []
            
            #REMOVE IGNORED_WORDS FOR QUILL VERSION:
            if word not in punct and word not in ignored_words:

                if idx + 2 <= len(sent_words):
                    #REMOVE IGNORED_WORDS FOR QUILL VERSION:
                    if sent_words[idx + 1] not in punct and sent_words[idx + 1] not in ignored_words:
                        bigrams.append(word + ' ' + sent_words[idx + 1])

                if idx + 3 <= len(sent_words):
                    #REMOVE IGNORED_WORDS FOR QUILL VERSION:
                    if sent_words[idx + 2] not in punct and sent_words[idx + 2] not in ignored_words:
                        trigrams.append(word + ' ' + sent_words[idx + 1]+ ' ' + sent_words[idx + 2])

                if idx + 4 <= len(sent_words):
                    #REMOVE IGNORED_WORDS FOR QUILL VERSION:
                    if sent_words[idx + 3] not in punct and sent_words[idx + 3] not in ignored_words:
                        four_grams.append(word + ' ' + sent_words[idx + 1]+ ' ' + sent_words[idx + 2]+ ' ' + sent_words[idx + 3])

                if idx + 5 <= len(sent_words):
                    #REMOVE IGNORED_WORDS FOR QUILL VERSION:
                    if sent_words[idx + 4] not in punct and sent_words[idx + 4] not in ignored_words:
                        five_grams.append(word + ' ' + sent_words[idx + 1]+ ' ' + sent_words[idx + 2]+ ' ' + sent_words[idx + 3]+ ' ' + sent_words[idx + 4])

                if idx + 6 <= len(sent_words):
                    #REMOVE IGNORED_WORDS FOR QUILL VERSION:
                    if sent_words[idx + 5] not in punct and sent_words[idx + 5] not in ignored_words:
                        six_grams.append(word + ' ' + sent_words[idx + 1]+ ' ' + sent_words[idx + 2]+ ' ' + sent_words[idx + 3]+ ' ' + sent_words[idx + 4]+ ' ' + sent_words[idx + 5])

                if idx + 7 <= len(sent_words):
                    #REMOVE IGNORED_WORDS FOR QUILL VERSION:
                    if sent_words[idx + 6] not in punct and sent_words[idx + 6] not in ignored_words:
                        seven_grams.append(word + ' ' + sent_words[idx + 1]+ ' ' + sent_words[idx + 2]+ ' ' + sent_words[idx + 3]+ ' ' + sent_words[idx + 4]+ ' ' + sent_words[idx + 5]+ ' ' + sent_words[idx + 6])

                if idx + 8 <= len(sent_words):
                    #REMOVE IGNORED_WORDS FOR QUILL VERSION:
                    if sent_words[idx + 7] not in punct and sent_words[idx + 7] not in ignored_words:
                        eight_grams.append(word + ' ' + sent_words[idx + 1]+ ' ' + sent_words[idx + 2]+ ' ' + sent_words[idx + 3]+ ' ' + sent_words[idx + 4]+ ' ' + sent_words[idx + 5]+ ' ' + sent_words[idx + 6]+ ' ' + sent_words[idx + 7])

                        
            package = {2 : bigrams,
                       3 : trigrams,
                       4 : four_grams,
                       5 : five_grams,
                       6 : six_grams,
                       7 : seven_grams,
                       8 : eight_grams}
            
            
            coll_matched = False

            n_gram_idx = 8

            while n_gram_idx >= 2:

                if coll_matched == False:

                    for coll in package[n_gram_idx]:
                        
                        coll = coll.lower()

                        #REMOVE FOR QUILL VERSION:
                        lemmed_coll = lem_coll(coll)

                        if lemmed_coll in n_gram_dict:
                            n_gram_dict[lemmed_coll]['Frequency'] += 1
                            
                            current_ex_sent_length = n_gram_dict[lemmed_coll]['Example sentence length']
                            
                            if current_ex_sent_length > len(sent_words):
                                n_gram_dict[lemmed_coll]['Original n-gram'] = coll
                                edited_coll = coll.replace(' , ', ', ')
                                n_gram_dict[lemmed_coll]['Example sentence'] = sent.replace(edited_coll, '<b>' + edited_coll + '</b>')
                                n_gram_dict[lemmed_coll]['Example sentence length'] = len(sent_words)
                            
                            coll_matched = True

                        else:
                            n_gram_dict[lemmed_coll] = {'Frequency' : 1, 'Original n-gram' : coll, 'Example sentence' : sent, 'Example sentence length' : len(sent_words)}

                n_gram_idx = n_gram_idx - 1
            
            idx += 1
        
        
        #print(n_gram_dict)
    
    #n_grams_list = list(n_gram_freqs.keys())
    #n_gram_freqs_list = list(n_gram_freqs.values())
    
    lemmed_n_grams_list = []
    original_n_grams_list = []
    ex_sents_list = []
    n_gram_freqs_list = []
    
    for key, value in n_gram_dict.items():
        lemmed_n_grams_list.append(key)
        original_n_grams_list.append(value['Original n-gram'])
        ex_sents_list.append(value['Example sentence'])
        n_gram_freqs_list.append(value['Frequency'])

    #print(package)
    
    #counted_package = count_n_grams(package)
    
    return {'Lemmed n-gram' : lemmed_n_grams_list, 'Original n-gram' : original_n_grams_list, 'Example sentence' : ex_sents_list, 'N-gram frequency' : n_gram_freqs_list}


In [ ]:
n_grams = extract_n_grams(directory)

In [ ]:
df = pd.DataFrame(n_grams).sort_values(by = ['N-gram frequency'], ascending = False)

In [ ]:
df = df[df['N-gram frequency']>= 2]

In [ ]:
df.shape

## Export n-gram freq list to csv:

In [ ]:
df.to_csv('/Users/markdecourcyling/Google Drive/Work, productivity and interests /computer science (technical)/coding skills (technical)/VS Code project files/NLP-powered Vocab Learning Strategy/Russian Vocab Project/n_grams_from_corpus.csv', '£')

## Import n-gram freq list from csv:

In [ ]:
df = pd.read_csv(proj_dir + '/n_grams_from_corpus.csv', '£')

In [28]:
def monoling_wikt_scrape(word):
    url = "https://en.wiktionary.org/w/index.php?title=" + word + '&printable=yes'
    response = requests.get(url)
    wikt_definitions = []
    try:
        soup = BeautifulSoup(response.text, "html.parser")
        if soup.find('ol') != None:
            entries = soup.find('ol')
            #if entries.findAll('li') != None:
            if entries.select('ol > li') != None:
                #number_of_defs = len(entries.findAll('li'))
                number_of_defs = len(entries.select('ol > li'))
                index = 0
                while index + 1 <= number_of_defs:
    
                    # definition:
                    entry = entries.select('ol > li')[index]
                    entry_text = entry.get_text()

                    if entry_text != '':

                        # register tag:           
                            
                        if entry.find('ul') != None:
                            sent_info_tag = entry.find('ul')
                            sent_info_text = sent_info_tag.get_text()
                            entry_text = entry_text.replace(sent_info_text, '')
                            
                            
                        ex_sent_text = ''
                        if entry.find('dd') != None:
                            ex_sent_tag = entry.find('dl')
                            ex_sent_text = ex_sent_tag.get_text()
                            entry_text = entry_text.replace(ex_sent_text, '')

                        capitalised_form = word.replace(word[0], word[0].upper())

                        ex_sent_with_blank = ex_sent_text.replace(word, '_____')
                        ex_sent_with_blank = ex_sent_with_blank.replace(capitalised_form, '_____')
                        ex_sent_full = ex_sent_text.replace(word, '<b>' + word + '</b>')
                        ex_sent_full = ex_sent_text.replace(capitalised_form, '<b>' + capitalised_form + '</b>')

                        definition_package = {'definition' : entry_text,
                                              'ex_sentence_blank' : ex_sent_with_blank,
                                                'ex_sentence_full' : ex_sent_full
                                             }

                        wikt_definitions.append(definition_package)
                    
                    index += 1
                    
                return(wikt_definitions)
            else:
                return('Error')
        else:
            return('Error')
    except Exception as e:
        return e

In [32]:
monoling_wikt_scrape('temerity')

[{'definition': '(uncountable) Reckless boldness; foolish bravery.\n\n',
  'ex_sentence_blank': 'Synonyms: audacity, foolhardiness, rashness, recklessness',
  'ex_sentence_full': 'Synonyms: audacity, foolhardiness, rashness, recklessness'},
 {'definition': '(countable) An act or case of reckless boldness.\n',
  'ex_sentence_blank': 'Draper, dear lad, had the illusion of an "intellectual sympathy" between them.... Draper\'s temerities would always be of that kind.',
  'ex_sentence_full': 'Draper, dear lad, had the illusion of an "intellectual sympathy" between them.... Draper\'s temerities would always be of that kind.'},
 {'definition': '(uncountable) Effrontery; impudence.\n\n',
  'ex_sentence_blank': 'Synonyms: brashness, cheek, gall, chutzpah',
  'ex_sentence_full': 'Synonyms: brashness, cheek, gall, chutzpah'}]

In [20]:
def skell_synonyms_scrape(word):
    base_url = "https://skell.sketchengine.co.uk/run.cgi/thesaurus?lpos=&query="
    response = requests.get("{url}/{endpoint}".format(url=base_url, endpoint=word))
    
    syns = []
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        if soup.find('div', {'id' : 'thes_content'}) != None:
            thes_content = soup.find('div', {'id' : 'thes_content'})
            
            syns = []
            
            syn_idx = 0
            
            while syn_idx + 1 <= 6:
                
                if thes_content.findAll('li')[syn_idx].get_text() != None:
                    syn = thes_content.findAll('li')[syn_idx].get_text()
                    syns.append(syn)
                
                syn_idx += 1
        else:
            synonyms_text = "No synonyms found."
    else:
        print('\tSynonym scraper couldn\'t connect to Skell for ' + word + '.')
        
    return(syns)

In [33]:
skell_synonyms_scrape('temerity')

['disingenuousness',
 'shortsightedness',
 'effrontery',
 'indigence',
 'improvidence',
 'obscurantism']

In [106]:
with open('/Users/markdecourcyling/Desktop/english_lit_vocab_flashcards.txt', 'w+') as file:
    
    test_recall_flashcards = open('/Users/markdecourcyling/Desktop/english_test_recall_flashcards.txt', 'w+') 
    
    with open('/Users/markdecourcyling/Desktop/english_lit_vocab.txt', 'r') as word_list:
        word_list_read = word_list.read()
        word_list = word_list_read.split('\n')
    
    word_list = list(set(word_list))
    
    language = 'english'

    parser = WiktionaryParser()
    
    for word in word_list:
        
        word = word.lower()

        if parser.fetch(word, language) != None:
            
            if parser.fetch(word, language):

                try:
                    data = parser.fetch(word, language)[0]
                    #time.sleep(2)

                    definitions = data['definitions'][0]['text']

                    item_idx = 0

                    def_list = definitions[1:]

                    for definition in def_list:

                        definitions_with_blank = ''
                        definitions_full = ''

                        for item in def_list:

                            if definition == item:

                                definitions_with_blank += '<b>' + '???????' + '</b><br><br>'

                                definitions_full += '<b>' + item + '</b><br><br>'

                            else:
                                definitions_with_blank += item + '<br><br>'

                                definitions_full += item + '<br><br>'


                        print(word + '£<br>' + definitions_with_blank + '£<br>' + definitions_full + '\n')

                        file.write(word + '£<br>' + definitions_with_blank + '£<br>' + definitions_full + '\n')
                        test_recall_flashcards.write(word + '£<br>' + definitions_full + '\n')

                        item_idx += 1
                
                except IndexError:
                    print('COULDN\'T WRITE CARDS FOR:')
                    print(word)
                    pass
                    
            else:
                continue
                    
        else:
            continue



COULDN'T WRITE CARDS FOR:

sociolect£<br><b>???????</b><br><br>£<br><b>(sociolinguistics) The variant of language used by a social group such as a socioeconomic class, an ethnic group, an age group, etc.</b><br><br>

beget£<br><b>???????</b><br><br>To cause; to produce.<br><br>To bring forth.<br><br>(Britain dialectal) To happen to; befall.<br><br>£<br><b>To father; to sire; to produce (a child).</b><br><br>To cause; to produce.<br><br>To bring forth.<br><br>(Britain dialectal) To happen to; befall.<br><br>

beget£<br>To father; to sire; to produce (a child).<br><br><b>???????</b><br><br>To bring forth.<br><br>(Britain dialectal) To happen to; befall.<br><br>£<br>To father; to sire; to produce (a child).<br><br><b>To cause; to produce.</b><br><br>To bring forth.<br><br>(Britain dialectal) To happen to; befall.<br><br>

beget£<br>To father; to sire; to produce (a child).<br><br>To cause; to produce.<br><br><b>???????</b><br><br>(Britain dialectal) To happen to; befall.<br><br>£<br>To fa

fractious£<br><b>???????</b><br><br>irritable; argumentative; quarrelsome<br><br>£<br><b>given to troublemaking</b><br><br>irritable; argumentative; quarrelsome<br><br>

fractious£<br>given to troublemaking<br><br><b>???????</b><br><br>£<br>given to troublemaking<br><br><b>irritable; argumentative; quarrelsome</b><br><br>

bucolic£<br><b>???????</b><br><br>Relating to the pleasant aspects of rustic country life.<br><br>Pertaining to herdsmen or peasants.<br><br>£<br><b>Rustic, pastoral, country-styled.</b><br><br>Relating to the pleasant aspects of rustic country life.<br><br>Pertaining to herdsmen or peasants.<br><br>

bucolic£<br>Rustic, pastoral, country-styled.<br><br><b>???????</b><br><br>Pertaining to herdsmen or peasants.<br><br>£<br>Rustic, pastoral, country-styled.<br><br><b>Relating to the pleasant aspects of rustic country life.</b><br><br>Pertaining to herdsmen or peasants.<br><br>

bucolic£<br>Rustic, pastoral, country-styled.<br><br>Relating to the pleasant aspects of rus

inevitability£<br><b>???????</b><br><br>(countable) An inevitable condition or outcome.<br><br>£<br><b>(uncountable) The condition of being inevitable.</b><br><br>(countable) An inevitable condition or outcome.<br><br>

inevitability£<br>(uncountable) The condition of being inevitable.<br><br><b>???????</b><br><br>£<br>(uncountable) The condition of being inevitable.<br><br><b>(countable) An inevitable condition or outcome.</b><br><br>

discourteous£<br><b>???????</b><br><br>£<br><b>impolite; lacking consideration for others</b><br><br>

reflective£<br><b>???????</b><br><br>Pondering, especially thinking back on the past.<br><br>(computing, programming) Involving reflection.<br><br>(grammar) Reciprocal.<br><br>£<br><b>That reflects, or redirects back to the source.</b><br><br>Pondering, especially thinking back on the past.<br><br>(computing, programming) Involving reflection.<br><br>(grammar) Reciprocal.<br><br>

reflective£<br>That reflects, or redirects back to the source.<br><br><b

logos£<br><b>???????</b><br><br>Alternative letter-case form of Logos<br><br>£<br><b>(rhetoric) A form of rhetoric in which the writer or speaker uses logic as the main argument.</b><br><br>Alternative letter-case form of Logos<br><br>

logos£<br>(rhetoric) A form of rhetoric in which the writer or speaker uses logic as the main argument.<br><br><b>???????</b><br><br>£<br>(rhetoric) A form of rhetoric in which the writer or speaker uses logic as the main argument.<br><br><b>Alternative letter-case form of Logos</b><br><br>

suffuse£<br><b>???????</b><br><br>(transitive, figuratively) To spread through or over in the manner of a liquid.<br><br>(transitive) To pour underneath.<br><br>£<br><b>(transitive) To spread through or over something, especially as a liquid, colour or light; to bathe.</b><br><br>(transitive, figuratively) To spread through or over in the manner of a liquid.<br><br>(transitive) To pour underneath.<br><br>

suffuse£<br>(transitive) To spread through or over something

endogenous£<br><b>???????</b><br><br>Of a natural process, or caused by factors within the body.<br><br>£<br><b>Produced, originating or growing from within.</b><br><br>Of a natural process, or caused by factors within the body.<br><br>

endogenous£<br>Produced, originating or growing from within.<br><br><b>???????</b><br><br>£<br>Produced, originating or growing from within.<br><br><b>Of a natural process, or caused by factors within the body.</b><br><br>

advocacy£<br><b>???????</b><br><br>the act of arguing in favour of, or supporting something<br><br>the practice of supporting someone to make their voice heard<br><br>£<br><b>the profession of an advocate</b><br><br>the act of arguing in favour of, or supporting something<br><br>the practice of supporting someone to make their voice heard<br><br>

advocacy£<br>the profession of an advocate<br><br><b>???????</b><br><br>the practice of supporting someone to make their voice heard<br><br>£<br>the profession of an advocate<br><br><b>the

...

tortuous£<br><b>???????</b><br><br>(astrology) Oblique; applied to the six signs of the zodiac (from Capricorn to Gemini) that ascend most rapidly and obliquely.<br><br>(obsolete) Injurious; tortious.<br><br>£<br><b>(often figuratively) Twisted; having many turns; convoluted.</b><br><br>(astrology) Oblique; applied to the six signs of the zodiac (from Capricorn to Gemini) that ascend most rapidly and obliquely.<br><br>(obsolete) Injurious; tortious.<br><br>

tortuous£<br>(often figuratively) Twisted; having many turns; convoluted.<br><br><b>???????</b><br><br>(obsolete) Injurious; tortious.<br><br>£<br>(often figuratively) Twisted; having many turns; convoluted.<br><br><b>(astrology) Oblique; applied to the six signs of the zodiac (from Capricorn to Gemini) that ascend most rapidly and obliquely.</b><br><br>(obsolete) Injurious; tortious.<br><br>

tortuous£<br>(often figuratively) Twisted; having many turns; convoluted.<br><br>(astrology) Oblique; applied to the six signs of the zodiac

balanced£<br><b>???????</b><br><br>£<br><b>simple past tense and past participle of balance</b><br><br>

ebullience£<br><b>???????</b><br><br>£<br><b>The quality of lively or enthusiastic expression of thoughts and feelings.</b><br><br>

bellicose£<br><b>???????</b><br><br>Showing or having the impulse to be combative.<br><br>£<br><b>Warlike in nature; aggressive; hostile.</b><br><br>Showing or having the impulse to be combative.<br><br>

bellicose£<br>Warlike in nature; aggressive; hostile.<br><br><b>???????</b><br><br>£<br>Warlike in nature; aggressive; hostile.<br><br><b>Showing or having the impulse to be combative.</b><br><br>

premise£<br><b>???????</b><br><br>(logic) Any of the first propositions of a syllogism, from which the conclusion is deduced.<br><br>(usually in the plural, law) Matters previously stated or set forth; especially, that part in the beginning of a deed, the office of which is to express the grantor and grantee, and the land or thing granted or conveyed, and a

multifarious£<br><b>???????</b><br><br>(law, of lawsuits) In which a party or a cause of action has been improperly or wrongfully joined together in the same suit, as in a misjoinder, perhaps as a result of a joinder of unrelated, distinct, independent parties or matters.<br><br>£<br><b>Having great diversity or variety; of various kinds; made up of many differing parts; manifold.</b><br><br>(law, of lawsuits) In which a party or a cause of action has been improperly or wrongfully joined together in the same suit, as in a misjoinder, perhaps as a result of a joinder of unrelated, distinct, independent parties or matters.<br><br>

multifarious£<br>Having great diversity or variety; of various kinds; made up of many differing parts; manifold.<br><br><b>???????</b><br><br>£<br>Having great diversity or variety; of various kinds; made up of many differing parts; manifold.<br><br><b>(law, of lawsuits) In which a party or a cause of action has been improperly or wrongfully joined together in

sardonic£<br><b>???????</b><br><br>Disdainfully or ironically humorous.<br><br>£<br><b>Scornfully mocking or cynical.</b><br><br>Disdainfully or ironically humorous.<br><br>

sardonic£<br>Scornfully mocking or cynical.<br><br><b>???????</b><br><br>£<br>Scornfully mocking or cynical.<br><br><b>Disdainfully or ironically humorous.</b><br><br>

genial£<br><b>???????</b><br><br>(especially of weather) Pleasantly mild and warm.<br><br>Marked by genius.<br><br>(archaic) Contributing to, or concerned in, propagation or production; generative; procreative; productive.<br><br>(obsolete) Belonging to one's genius or natural character; native; natural; inborn.<br><br>£<br><b>Friendly and cheerful.</b><br><br>(especially of weather) Pleasantly mild and warm.<br><br>Marked by genius.<br><br>(archaic) Contributing to, or concerned in, propagation or production; generative; procreative; productive.<br><br>(obsolete) Belonging to one's genius or natural character; native; natural; inborn.<br><br>

gen

paucity£<br><b>???????</b><br><br>A smallness in size or amount that is insufficient; meagerness, dearth.<br><br><br><br>£<br><b>Fewness in number; too few.</b><br><br>A smallness in size or amount that is insufficient; meagerness, dearth.<br><br><br><br>

paucity£<br>Fewness in number; too few.<br><br><b>???????</b><br><br><br><br>£<br>Fewness in number; too few.<br><br><b>A smallness in size or amount that is insufficient; meagerness, dearth.</b><br><br><br><br>

paucity£<br>Fewness in number; too few.<br><br>A smallness in size or amount that is insufficient; meagerness, dearth.<br><br><b>???????</b><br><br>£<br>Fewness in number; too few.<br><br>A smallness in size or amount that is insufficient; meagerness, dearth.<br><br><b></b><br><br>

conversant£<br><b>???????</b><br><br>familiar or acquainted by use or study; well-informed; versed<br><br>(obsolete) Concerned; occupied.<br><br>£<br><b>closely familiar; current; having frequent interaction</b><br><br>familiar or acquainted by u

In [78]:
with open('/Users/markdecourcyling/Desktop/english_lit_vocab_flashcards.txt', 'w+') as file:
    
    with open('/Users/markdecourcyling/Desktop/english_lit_vocab.txt', 'r') as word_list:
        word_list_read = word_list.read()
        word_list = word_list_read.split('\n')
    
    for word in word_list:
        dict_defs = monoling_wikt_scrape(word)
        
        print(dict_defs)
        
        if dict_defs != 'Error':

            item_idx = 0
            for dict_definition in dict_defs:
                
                monoling_defs_blank = ''
                monoling_defs_complete = ''
                
                for item in dict_defs:

                    definition = dict_definition['definition']
                    ex_sent_blank = dict_definition['ex_sentence_blank']
                    ex_sent_full = dict_definition['ex_sentence_full']

                    if dict_defs.index(item) == item_idx:
                        monoling_defs_blank += '<b>' + '???????' + '<br><emsp>' + ex_sent_blank + '</b><br><br>'

                        monoling_defs_complete += '<b>' + definition + '<br><emsp>' + ex_sent_blank + '</b><br><br>'

                    else:
                        monoling_defs_blank += definition + '<br><emsp>' + ex_sent_blank + '<br><br>'

                        monoling_defs_complete += definition + '<br><emsp>' + ex_sent_full + '<br><br>'


                    print(word + '£<br>' + monoling_defs_blank + '£<br>' + monoling_defs_complete + '\n')

                    file.write(word + '£<br>' + monoling_defs_blank + '£<br>' + monoling_defs_complete + '\n')

                    item_idx += 1
                    
    
    
monoling_wikt_scrape('admonition')

aberration (deviation)




(optics) aberration




(physiology) aberration




[{'definition': 'aberration (deviation)', 'ex_sentence_blank': '', 'ex_sentence_full': ''}, {'definition': '(optics) aberration', 'ex_sentence_blank': '', 'ex_sentence_full': ''}, {'definition': '(physiology) aberration', 'ex_sentence_blank': '', 'ex_sentence_full': ''}]
Aberration£<br><b>???????<br><emsp></b><br><br>£<br><b>aberration (deviation)<br><emsp></b><br><br>

Aberration£<br><b>???????<br><emsp></b><br><br><b>???????<br><emsp></b><br><br>£<br><b>aberration (deviation)<br><emsp></b><br><br><b>aberration (deviation)<br><emsp></b><br><br>

Aberration£<br><b>???????<br><emsp></b><br><br><b>???????<br><emsp></b><br><br><b>???????<br><emsp></b><br><br>£<br><b>aberration (deviation)<br><emsp></b><br><br><b>aberration (deviation)<br><emsp></b><br><br><b>aberration (deviation)<br><emsp></b><br><br>

Aberration£<br>(optics) aberration<br><emsp><br><br>£<br>(optics) aberration<br><emsp><br><br>

Aberration£<b

(chiefly in the plural) Patronage or protection.
This building was built under the auspices of the Friends of the Poor.
FOUND EX SENT
This building was built under the auspices of the Friends of the Poor.
This building was built under the _____s of the Friends of the Poor.



An omen or a sign.
The circle of vultures was not a good auspice.
FOUND EX SENT
The circle of vultures was not a good auspice.
The circle of vultures was not a good _____.



(obsolete) Divination from the actions of birds.




[{'definition': '(chiefly in the plural) Patronage or protection.\n', 'ex_sentence_blank': 'This building was built under the _____s of the Friends of the Poor.', 'ex_sentence_full': 'This building was built under the auspices of the Friends of the Poor.'}, {'definition': 'An omen or a sign.\n', 'ex_sentence_blank': 'The circle of vultures was not a good _____.', 'ex_sentence_full': 'The circle of vultures was not a good auspice.'}, {'definition': '(obsolete) Divination from the actions of 

Error
Error
(transitive, intransitive) To reject disdainfully; contemn; scorn.
Shakespeare
to spurn at your most royal image
Shakespeare
What safe and nicely I might well delay / By rule of knighthood, I disdain and spurn.
John Locke
Domestics will pay a more cheerful service when they find themselves not spurned because fortune has laid them at their master's feet.
FOUND EX SENT
to spurn at your most royal image
to _____ at your most royal image



(transitive) To reject something by pushing it away with the foot.
Shakespeare
I spurn thee like a cur out of my way.
FOUND EX SENT
I spurn thee like a cur out of my way.
I _____ thee like a cur out of my way.



(transitive) To waste; fail to make the most of (an opportunity)
2011 September 28,  Tom Rostance,  “Arsenal 2 - 1 Olympiakos”, in  BBC Sport‎[1]:Marouane Chamakh then spurned a great chance to kill the game off when he ran onto Andrey Arshavin's lofted through ball but shanked his shot horribly across the face of goal.
FOUND EX SE

tight-lippedness, discretion, avoidance of saying too much




a silent and reserved nature




[{'definition': 'tight-lippedness, discretion, avoidance of saying too much', 'ex_sentence_blank': '', 'ex_sentence_full': ''}, {'definition': 'a silent and reserved nature', 'ex_sentence_blank': '', 'ex_sentence_full': ''}]
reticence£<br><b>???????<br><emsp></b><br><br>£<br><b>tight-lippedness, discretion, avoidance of saying too much<br><emsp></b><br><br>

reticence£<br><b>???????<br><emsp></b><br><br><b>???????<br><emsp></b><br><br>£<br><b>tight-lippedness, discretion, avoidance of saying too much<br><emsp></b><br><br><b>tight-lippedness, discretion, avoidance of saying too much<br><emsp></b><br><br>

reticence£<br>a silent and reserved nature<br><emsp><br><br>£<br>a silent and reserved nature<br><emsp><br><br>

reticence£<br>a silent and reserved nature<br><emsp><br><br>a silent and reserved nature<br><emsp><br><br>£<br>a silent and reserved nature<br><emsp><br><br>a silent and reserved 

Error
The property of seeming true, of resembling reality; resemblance to reality, realism.




A statement which merely appears to be true.




(fiction) Faithfulness to its own rules; internal cohesion.




[{'definition': 'The property of seeming true, of resembling reality; resemblance to reality, realism.', 'ex_sentence_blank': '', 'ex_sentence_full': ''}, {'definition': 'A statement which merely appears to be true.', 'ex_sentence_blank': '', 'ex_sentence_full': ''}, {'definition': '(fiction) Faithfulness to its own rules; internal cohesion.', 'ex_sentence_blank': '', 'ex_sentence_full': ''}]
verisimilitude£<br><b>???????<br><emsp></b><br><br>£<br><b>The property of seeming true, of resembling reality; resemblance to reality, realism.<br><emsp></b><br><br>

verisimilitude£<br><b>???????<br><emsp></b><br><br><b>???????<br><emsp></b><br><br>£<br><b>The property of seeming true, of resembling reality; resemblance to reality, realism.<br><emsp></b><br><br><b>The property of seeming tr

KeyboardInterrupt: 

In [ ]:
def wiktionary_parser_eng_trans_scrape(word):
    language = 'english'

    parser = WiktionaryParser()
    if parser.fetch(word, language) != None:
        if parser.fetch(word, language) != None:
            data = parser.fetch(word, language)
            try:
                definitions = data[0]['definitions']  
                try:
                    definitions_list = definitions[0]['text']
                    definitions_list = definitions_list[1:]
                    for definition in definitions_list:
                        if 'Alternative spelling of ' in definition:
                            alt_spelling = definition.split('Alternative spelling of ')[1]
                            # print(alt_spelling.split())
                            # print(alt_spelling.split()[0])
                            alt_spelling = alt_spelling.split()[0]
                            # print(alt_spelling)
                            if parser.fetch(alt_spelling, language) != None:
                                    data = parser.fetch(alt_spelling, language)
                                    definitions = data[0]['definitions']
                                    print('Wiktionary scraper found the alt spelling \'' + alt_spelling + '\' and returned its definitions.')
                                    if definitions:
                                        definitions_list = definitions[0]['text']
                                        return_list = definitions_list[1:]
                                    else:
                                        return_list = ['WiktionaryParser can\'t find definitions for \'' + alt_spelling + '\'.']
                        else:
                            return_list = definitions_list
                            
                except Exception:
                    return_list = ['No', 'translations', 'found']
            except Exception:
                return_list = ['No', 'translations', 'found']
        else:
            # complete_definitions_text = 'WiktionaryParser can\'t find definitions for \'' + word + '\'.'
            return_list = ['No', 'translations', 'found']
    else:
        # complete_definitions_text = 'WiktionaryParser can\'t find definitions for \'' + word + '\'.'
        return_list = ['No', 'translations', 'found']
    
    return(return_list)

In [7]:
def old_coll_search(term):
    
    base_url = 'https://api.sketchengine.eu/bonito/run.cgi'

    data_for_wsketch = {
        'corpname': 'preloaded/bnc2',
        'format': 'json',
        'username': 'spet4310',
        'api_key': '1306d2a48d4d4a61861700bdaa5403db'
    }
     
    word_data ={}
    
    grammatical_collocates = []
    lexical_collocates = []

    data_for_wsketch['lemma'] = term
    wsketch_status_code = requests.get(base_url + '/wsketch', params=data_for_wsketch).status_code
    print('Wordsketch request status code:', wsketch_status_code)
    if wsketch_status_code == 429:
        sys.exit()
    elif wsketch_status_code == 200:
        wsketch_data = requests.get(base_url + '/wsketch', params=data_for_wsketch).json()
        time.sleep(2)
        
        if 'Gramrels' in wsketch_data.keys():
            for g in wsketch_data['Gramrels'][:10]:
                
                gram_coll_groups = ['prec_prep', 'post_prep']
            
                if g['name'] in gram_coll_groups:
                    
                    for i in g['Words'][:10]:
                        
                        if i['score'] >= 4.5 and i['count'] >= 15:
                            # pp.pprint(i)
                            # -------- find the concordances of the collocation
                            seek_number = i['filterseek']
                            data_for_wsketch['q'] = str('w' + seek_number)
                            data_for_wsketch['gdex_enabled'] = 1
                            # data_str = requests.get(base_url + '/view', params=view_data).json()
                            view_status_code = requests.get('https://api.sketchengine.eu/bonito/run.cgi/view?corpname=preloaded/bnc2;format=json;username=markdecl;api_key=ba5d440bae8642ffa65cabc2d1f7a6f0;q=w' + seek_number + ';gdex_enabled=1;pagesize=100').status_code
                            # print('View request status_code:', view_status_code)
                            if view_status_code == 200:
                                data_str = requests.get('https://api.sketchengine.eu/bonito/run.cgi/view?corpname=preloaded/bnc2;format=json;username=markdecl;api_key=ba5d440bae8642ffa65cabc2d1f7a6f0;q=w' + seek_number + ';gdex_enabled=1;pagesize=100;viewmode=\'sen\'').json()
                                time.sleep(3)
                                # pp.pprint(data_str)
                                a = data_str['Lines']
                                number_of_lines = data_str['Desc'][0]['size']
                                line_no = 0
                                
                                # print(a)
                                
                                collocation, ex_sent_with_blank, ex_sent = gc_ex_sent(a, line_no)
                                
                                coll = i['cm']
                                
                                # print(ex_sent)

                                # --------- ADD COLLOCATION PAPER FREQ DIST HERE

                                grammatical_collocates.append([i['word'], i['count'], ex_sent_with_blank, ex_sent, coll])
                                line_no += 1
                else:
                    lex_coll_found = 'n'
                    for i in g['Words'][:10]:
                        # print(i)
                        if i['score'] >= 8.5 and i['count'] >= 6:
                                # pp.pprint(i)
                                lex_coll_found = 'y'
                                # -------- find the concordances of the collocation
                                seek_number = i['filterseek']
                                data_for_wsketch['q'] = str('w' + seek_number)
                                data_for_wsketch['gdex_enabled'] = 1

                                view_status_code = requests.get('https://api.sketchengine.eu/bonito/run.cgi/view?corpname=preloaded/bnc2;format=json;username=markdecl;api_key=ba5d440bae8642ffa65cabc2d1f7a6f0;q=w' + seek_number + ';gdex_enabled=1;pagesize=100').status_code

                                if view_status_code == 200:
                                    data_str = requests.get('https://api.sketchengine.eu/bonito/run.cgi/view?corpname=preloaded/bnc2;format=json;username=markdecl;api_key=ba5d440bae8642ffa65cabc2d1f7a6f0;q=w' + seek_number + ';gdex_enabled=1;pagesize=100;viewmode=sen').json()
                                    time.sleep(3)
                                    a = data_str['Lines']
                                    number_of_lines = data_str['Desc'][0]['size']
                                    line_no = 0
                                    
                                    # pp.pprint(a)
                                    
                                    collocation, ex_sent_with_blank, ex_sent = lc_ex_sent(a, line_no)
                                    
                                    coll = i['cm']

                                    # print(ex_sent)

                                    lexical_collocates.append([i['word'], i['count'], ex_sent_with_blank, ex_sent, coll])
                                    line_no += 1  
                    
                    if lex_coll_found == 'n':
                        
                        # ----------- look for the collocation with the next highest frequency
                        highest_freq = 0
                        for i in g['Words'][:10]:
                            if i['count'] > highest_freq:
                                highest_freq_coll = i
                        print(highest_freq_coll)
                        seek_number = highest_freq_coll['filterseek']
                        data_for_wsketch['q'] = str('w' + seek_number)
                        data_for_wsketch['gdex_enabled'] = 1
                        # data_str = requests.get(base_url + '/view', params=view_data).json()
                        view_status_code = requests.get('https://api.sketchengine.eu/bonito/run.cgi/view?corpname=bnc;format=json;username=markdecl;api_key=ba5d440bae8642ffa65cabc2d1f7a6f0;q=w' + seek_number + ';gdex_enabled=1;pagesize=100').status_code
                        # print('View request status_code:', view_status_code)
                        if view_status_code == 200:
                            data_str = requests.get('https://api.sketchengine.eu/bonito/run.cgi/view?corpname=bnc;format=json;username=markdecl;api_key=ba5d440bae8642ffa65cabc2d1f7a6f0;q=w' + seek_number + ';gdex_enabled=1;pagesize=100;viewmode=sen').json()
                            time.sleep(3)
                            a = data_str['Lines']
                            number_of_lines = data_str['Desc'][0]['size']
                            line_no = 0
                            
                            # print(a)
                           
                            collocation, ex_sent_with_blank, ex_sent = lc_ex_sent(a, line_no)
                            
                            coll = i['cm']
                            
                            lexical_collocates.append([highest_freq_coll['word'], highest_freq_coll['count'], ex_sent_with_blank, ex_sent, coll])

    word_data['grammatical collocates'] = grammatical_collocates
    word_data['lexical collocates'] = lexical_collocates
    
    # print(lexical_collocates)
    
    return word_data

In [8]:
old_coll_search('intense')

Wordsketch request status code: 200
{'count': 24, 'word': 'Sing', 'name': 'Sing', 'conclink': '[ws(2, 46087996480)]', 'score': 1.05, 'Words': [], 'headword': 'intense', 'seek': '46087996480'}


KeyError: 'filterseek'